In [1]:
import pandas as pd
import glob

In [2]:
## MAIN FUNCTION - input

## INPUT FILE
PPP = True
if PPP:
    input_folder = 'PPP2017_updated_Nov8_2021_Accessed_Apr2022/*.xls'
    ## OUTPUT FILE
    save_filename = 'physical_ppp.csv'
else:
    input_folder = 'physical/*.xls'
    ## OUTPUT FILE
    save_filename = 'physical.csv'

# change the name from world bank names to common names
countries = pd.read_csv('../../data/dl1_countrycodeorg_country_name.csv')
country_names = countries["capital country name"].unique()
country_map = dict(zip(countries['capital country name'], countries['Country Code']))



In [3]:
pieces = []
for i, file in enumerate(glob.glob(input_folder)):
    df = pd.read_excel(file, skiprows=7, header=None)
    des = df.loc[0, 1]
    country = des.split(',')[0].split('for ')[1]
    if country == 'Taiwan Province of China':
        country = 'Province of China Taiwan'
    if not((country in country_names) ):
        print(i, country)
    try:
        df = pd.read_excel(file, skiprows=11, header=None)
        df = df.rename(columns={0: 'time', 1: 'value'})
        df['Country Code'] = country_map[country]
        pieces.append(df)
    except:
        print('%s'%(country))
print(i+1)

df = pd.concat(pieces)
df['time'] = df['time'].apply(lambda x: str(x.year))
df = df[df['time']>='2010']

df = pd.pivot_table(df, values='value', index='Country Code', columns = 'time')
df.to_csv(save_filename)

167
